In [2]:
import numpy as np
import cv2 as cv
import va
from ipywidgets import interact

In [5]:
video = cv.VideoCapture("./simpsons abbey road.mp4")
prev = None
fm = []
thr = 50
while True:
    ret, frame = video.read()
    if not ret:
        # No more frames
        break
    f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY).astype(np.uint16)
    if prev is not None:
        mask = np.zeros_like(f)
        mask[np.abs(f - prev) > thr] = 255
        fm.append((frame, mask))
    prev = f
video.release()


In [6]:
@interact(i=(0,len(fm)-1))
def frameVision(i=0):
    va.show(fm[i][1])


interactive(children=(IntSlider(value=0, description='i', max=241), Output()), _dom_classes=('widget-interact'…

In [7]:
mog = cv.createBackgroundSubtractorMOG2(detectShadows=False)

video = cv.VideoCapture("./simpsons abbey road.mp4")
fm = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    mask = mog.apply(f)
    fm.append((frame, mask))
video.release()

In [8]:
@interact(i=(0,len(fm)-1))
def frameVisionMOG(i=0):
    va.show(fm[i][1])


interactive(children=(IntSlider(value=0, description='i', max=242), Output()), _dom_classes=('widget-interact'…

## Improve the FRAME
1. Resize del frame in modo da mantenere il focus solamente sull'oggetto in movimento

In [12]:
MASK_SF = 2.0

video = cv.VideoCapture("./simpsons abbey road.mp4")
fm1 = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    mask = mog.apply(cv.resize(f, None, fx=1/MASK_SF, fy=1/MASK_SF))
    n_mask = cv.resize(mask, None, fx=MASK_SF, fy=MASK_SF, interpolation=cv.INTER_NEAREST)
    fm1.append((frame, mask))
video.release()


In [13]:
@interact(i=(0,len(fm1)-1))
def frameVisionMOG2(i=0):
    va.show(fm1[i][1])


interactive(children=(IntSlider(value=0, description='i', max=242), Output()), _dom_classes=('widget-interact'…

2. Utilizzo delle componenti connesse per eliminare tutte le componenti con area minore di una certa soglia

In [20]:
MASK_SF = 2.0

video = cv.VideoCapture("./simpsons abbey road.mp4")
fm2 = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    mask = mog.apply(cv.resize(f, None, fx=1/MASK_SF, fy=1/MASK_SF))
    n, cc, stats, _ = cv.connectedComponentsWithStats(mask)
    small = [i for i in range(1, n) if stats[i, cv.CC_STAT_AREA] < 70]
    mask[np.isin(cc, small)] = 0
    n_mask = cv.resize(mask, None, fx=MASK_SF, fy=MASK_SF, interpolation=cv.INTER_NEAREST)
    fm2.append((frame, mask))
video.release()


In [21]:
@interact(i=(0,len(fm1)-1))
def frameVisionMOG2(i=0):
    va.show(fm2[i][1])


interactive(children=(IntSlider(value=0, description='i', max=242), Output()), _dom_classes=('widget-interact'…

3. Riempio i buchi all'interno delle componenti connesse

In [38]:
MASK_SF = 2.0
se = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
video = cv.VideoCapture("./simpsons abbey road.mp4")
fm3 = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    mask = mog.apply(cv.resize(f, None, fx=1/MASK_SF, fy=1/MASK_SF))
    
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, se)
    n, cc, stats, _ = cv.connectedComponentsWithStats(mask, connectivity=4)
    small = [i for i in range(1, n) if stats[i, cv.CC_STAT_AREA] < 70]
    mask[np.isin(cc, small)] = 0
    
    n, cc, stats, _ = cv.connectedComponentsWithStats(mask, connectivity=4)
    holes = [i for i in range(1, n) if stats[i, cv.CC_STAT_AREA] < 2000]
    mask[np.isin(cc, holes)] = 255
    
    
    n_mask = cv.resize(mask, None, fx=MASK_SF, fy=MASK_SF, interpolation=cv.INTER_NEAREST)
    fm3.append((frame, mask))
video.release()


In [39]:
@interact(i=(0,len(fm1)-1))
def frameVisionMOG2(i=0):
    va.show(fm3[i][1])


interactive(children=(IntSlider(value=0, description='i', max=242), Output()), _dom_classes=('widget-interact'…

## Mean SHIFT